<a href="https://colab.research.google.com/github/samira670/Application-Task-Manager-RestApi/blob/main/CvT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchvision --upgrade

In [ ]:
!pip install timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm

# Set the path to your dataset
train_path = '/content/drive/MyDrive/Dataset/Train'
test_path = '/content/drive/MyDrive/Dataset/Test'

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transforms for training and testing data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the datasets with the ImageFolder function
image_datasets = {
    'train': datasets.ImageFolder(train_path, transform=data_transforms['train']),
    'test': datasets.ImageFolder(test_path, transform=data_transforms['test']),
}

# Define the dataloaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=4),
}

# Get the number of classes in the dataset
num_classes = len(image_datasets['train'].classes)

# Load a pre-trained Convolutional Vision Transformer (CVT) model
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# Replace the last fully connected layer for the new classification task
model.head = nn.Linear(model.head.in_features, num_classes)

# Move the model to the appropriate device
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
def train_model(model, criterion, optimizer, num_epochs=10):
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(image_datasets['train'])
        train_acc = running_corrects.double() / len(image_datasets['train'])
        print(f"Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f}")

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0

        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data)

        val_loss = val_running_loss / len(image_datasets['test'])
        val_acc = val_running_corrects.double() / len(image_datasets['test'])
        print(f"Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        print()

# Train the model
num_epochs = 10
train_model(model, criterion, optimizer, num_epochs)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/10
----------
Train Loss: 0.5424 Train Acc: 0.7347
Val Loss: 0.3633 Val Acc: 0.8357

Epoch 2/10
----------
Train Loss: 0.3482 Train Acc: 0.8510
Val Loss: 0.2460 Val Acc: 0.8714

Epoch 3/10
----------
Train Loss: 0.3406 Train Acc: 0.8531
Val Loss: 0.2656 Val Acc: 0.8857

Epoch 4/10
----------
Train Loss: 0.2443 Train Acc: 0.9010
Val Loss: 0.2906 Val Acc: 0.8929

Epoch 5/10
----------
Train Loss: 0.3667 Train Acc: 0.8571
Val Loss: 0.2705 Val Acc: 0.8643

Epoch 6/10
----------
Train Loss: 0.2229 Train Acc: 0.9224
Val Loss: 0.2280 Val Acc: 0.8786

Epoch 7/10
----------
Train Loss: 0.1644 Train Acc: 0.9357
Val Loss: 0.1826 Val Acc: 0.9429

Epoch 8/10
----------
Train Loss: 0.1470 Train Acc: 0.9469
Val Loss: 0.4213 Val Acc: 0.8214

Epoch 9/10
----------
Train Loss: 0.1662 Train Acc: 0.9347
Val Loss: 0.3227 Val Acc: 0.8929

Epoch 10/10
----------
Train Loss: 0.2051 Train Acc: 0.9245
Val Loss: 0.1376 Val Acc: 0.9571

